In [1]:
!pip3 install -U pip
!pip3 install -U setuptools wheel
!pip3 install --pre autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.1 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pk

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
#load
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


In [3]:
label = 'class'
print("Summary of class variable: \n", train_data[label].describe())
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data) #train

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass/"
AutoGluon Version:  0.5.2b20220725
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    500
Train Data Columns: 14
Label Column: class
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preprocess the data ...
Fi

Summary of class variable: 
 count        500
unique         2
top        <=50K
freq         365
Name: class, dtype: object


			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('int', [])    : 6 | ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', ...]
		('object', []) : 8 | ['workclass', 'education', 'marital-status', 'occupation', 'relationship', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('category', [])  : 7 | ['workclass', 'education', 'marital-status', 'occupation', 'relationship', ...]
		('int', [])       : 6 | ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', ...]
		('int', ['bool']) : 1 | ['sex']
	0.2s = Fit runtime
	14 features in original data used to generate 14 features in processed data.
	Train Data (Processed) Memory Usage: 0.03 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.26s ...
AutoGluon will gauge predictive performance using evaluation metric: 'accuracy'
	To c

In [4]:
# test
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


In [5]:
#predict
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.8374449790152523
Evaluations on test data:
{
    "accuracy": 0.8374449790152523,
    "balanced_accuracy": 0.7430558394221018,
    "mcc": 0.5243657567117436,
    "f1": 0.621904761904762,
    "precision": 0.69394261424017,
    "recall": 0.5634167385677308
}


Predictions:  
 0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object


In [6]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.842973,0.84,0.322512,0.226139,1.362021,0.322512,0.226139,1.362021,1,True,5
1,CatBoost,0.842461,0.85,0.024925,0.012429,2.191962,0.024925,0.012429,2.191962,1,True,7
2,RandomForestEntr,0.841130,0.83,0.322906,0.111239,1.149384,0.322906,0.111239,1.149384,1,True,6
3,LightGBM,0.839799,0.85,0.065234,0.020216,0.355319,0.065234,0.020216,0.355319,1,True,4
4,XGBoost,0.837445,0.87,0.065790,0.009371,0.311308,0.065790,0.009371,0.311308,1,True,11
5,WeightedEnsemble_L2,0.837445,0.87,0.069187,0.010133,0.786526,0.003397,0.000762,0.475218,2,True,14
6,LightGBMXT,0.836421,0.83,0.024642,0.050305,1.076967,0.024642,0.050305,1.076967,1,True,3
7,ExtraTreesGini,0.833453,0.82,0.327553,0.107891,0.747643,0.327553,0.107891,0.747643,1,True,8
8,ExtraTreesEntr,0.832839,0.81,0.325423,0.208226,0.738235,0.325423,0.208226,0.738235,1,True,9
9,LightGBMLarge,0.828949,0.83,0.055618,0.010657,0.404541,0.055618,0.010657,0.404541,1,True,13


# Description of fit():

In [7]:
pred_probs = predictor.predict_proba(test_data_nolab)
pred_probs.head(5)

,<=50K,>50K
0,0.982107,0.017893
1,0.988337,0.011663
2,0.573505,0.426495
3,0.998272,0.001728
4,0.990299,0.009701


In [8]:
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0               XGBoost       0.87       0.009371  0.311308                0.009371           0.311308            1       True         11
1   WeightedEnsemble_L2       0.87       0.010133  0.786526                0.000762           0.475218            2       True         14
2              CatBoost       0.85       0.012429  2.191962                0.012429           2.191962            1       True          7
3        NeuralNetTorch       0.85       0.015212  2.896114                0.015212           2.896114            1       True         12
4              LightGBM       0.85       0.020216  0.355319                0.020216           0.355319            1       True          4
5      RandomForestGini       0.84       0.226139  1.362021                0.226139           1.362021        

In [9]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  binary
AutoGluon identified the following types of features:
('category', [])  : 7 | ['workclass', 'education', 'marital-status', 'occupation', 'relationship', ...]
('int', [])       : 6 | ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', ...]
('int', ['bool']) : 1 | ['sex']


In [10]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestGini,0.842973,0.84,0.322815,0.226139,1.362021,0.322815,0.226139,1.362021,1,True,5
1,CatBoost,0.842461,0.85,0.021961,0.012429,2.191962,0.021961,0.012429,2.191962,1,True,7
2,RandomForestEntr,0.841130,0.83,0.320083,0.111239,1.149384,0.320083,0.111239,1.149384,1,True,6
3,LightGBM,0.839799,0.85,0.049772,0.020216,0.355319,0.049772,0.020216,0.355319,1,True,4
4,XGBoost,0.837445,0.87,0.084594,0.009371,0.311308,0.084594,0.009371,0.311308,1,True,11
5,WeightedEnsemble_L2,0.837445,0.87,0.089202,0.010133,0.786526,0.004608,0.000762,0.475218,2,True,14
6,LightGBMXT,0.836421,0.83,0.023109,0.050305,1.076967,0.023109,0.050305,1.076967,1,True,3
7,ExtraTreesGini,0.833453,0.82,0.326217,0.107891,0.747643,0.326217,0.107891,0.747643,1,True,8
8,ExtraTreesEntr,0.832839,0.81,0.325503,0.208226,0.738235,0.325503,0.208226,0.738235,1,True,9
9,LightGBMLarge,0.828949,0.83,0.090425,0.010657,0.404541,0.090425,0.010657,0.404541,1,True,13


# Regression (predicting numeric table columns):

In [11]:
age_column = 'age'
print("Summary of age variable: \n", train_data[age_column].describe())

Summary of age variable: 
 count    500.00000
mean      39.65200
std       13.52393
min       17.00000
25%       29.00000
50%       38.00000
75%       49.00000
max       85.00000
Name: age, dtype: float64


In [12]:
predictor_age = TabularPredictor(label=age_column, path="agModels-predictAge").fit(train_data, time_limit=60)
performance = predictor_age.evaluate(test_data)

Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "agModels-predictAge/"
AutoGluon Version:  0.5.2b20220725
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    500
Train Data Columns: 14
Label Column: age
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (85, 17, 39.652, 13.52393)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11790.98 MB
	Train Data (Original)  Memory Usage: 0.32 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually sp

In [13]:
predictor_age.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-10.496803,-11.208568,1.471395,0.146620,4.219767,0.006260,0.000855,0.697830,2,True,12
1,ExtraTreesMSE,-10.656025,-11.369094,0.551919,0.107647,0.735253,0.551919,0.107647,0.735253,1,True,7
2,RandomForestMSE,-10.745634,-11.666909,0.538059,0.107841,0.843101,0.538059,0.107841,0.843101,1,True,5
3,CatBoost,-10.780312,-11.799279,0.043658,0.011373,1.587199,0.043658,0.011373,1.587199,1,True,6
4,LightGBMXT,-10.837373,-11.709228,0.430785,0.012718,0.385737,0.430785,0.012718,0.385737,1,True,3
5,LightGBM,-10.972156,-11.929546,0.105052,0.012788,0.253614,0.105052,0.012788,0.253614,1,True,4
6,XGBoost,-11.115033,-12.289224,0.085566,0.009595,0.444444,0.085566,0.009595,0.444444,1,True,9
7,NeuralNetFastAI,-11.225699,-12.073283,0.269424,0.018449,0.820539,0.269424,0.018449,0.820539,1,True,8
8,NeuralNetTorch,-11.448391,-11.995427,0.396866,0.015805,1.956503,0.396866,0.015805,1.956503,1,True,10
9,LightGBMLarge,-11.469922,-12.315314,0.243182,0.010526,0.443809,0.243182,0.010526,0.443809,1,True,11
